In [16]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.market.curves import *
import datetime as dt

In [17]:
swap_cal_type = CalendarTypes.TARGET
bd_type = BusDayAdjustTypes.FOLLOWING
dg_type = DateGenRuleTypes.BACKWARD


fixed_cpn = 0.034
fixed_freq_type = FrequencyTypes.QUARTERLY
fixed_dc_type = DayCountTypes.THIRTY_360_BOND

float_spread = 0.0
float_freq_type = FrequencyTypes.QUARTERLY
float_dc_type = DayCountTypes.THIRTY_360_BOND

swap_type = SwapTypes.PAY
notional = 60000000

start_dt = Date(1, 4, 2024)
maturity_dt = start_dt.add_years(1.75)

swap = IborSwap(start_dt,
                maturity_dt,
                swap_type,
                fixed_cpn,
                fixed_freq_type,
                fixed_dc_type,
                notional,
                float_spread,
                float_freq_type,
                float_dc_type,
                swap_cal_type,
                bd_type,
                dg_type)


In [18]:
value_date= dt.datetime(2024,3,31)
value_dt = from_datetime(value_date)

settle_dt = value_dt.add_weekdays(0)
# settle_dt=value_dt
settle_dt

31-MAR-2024

In [19]:
# 书中曲线
# yield_curve=[i/100 for i in [0.75,1.1853,1.6822,2.1474,2.4371,2.7390,3.0198]]
# zero_dts=value_dt.add_years([0.25,0.5,0.75,1.0,1.25,1.5,1.75])


yield_curve=[i/100 for i in [0.75,1.1853,1.6822,2.1474,2.4371,2.7390,3.0198]]
zero_dts=value_dt.add_years([0.25,0.5,0.75,1.0,1.25,1.5,1.75])

zero_curve = DiscountCurveZeros(value_dt=value_dt,
                                zero_dts=zero_dts,
                                zero_rates=yield_curve,
                                freq_type=FrequencyTypes.QUARTERLY,
                                dc_type=DayCountTypes.THIRTY_360_BOND,
                                interp_type=InterpTypes.FINCUBIC_ZERO_RATES
                                )

In [20]:
swap.swap_rate(value_dt, zero_curve)

0.030065619962567392

In [21]:
# def payer_side_macaulay_duration(value_dt, discount_curve):
#     """Calculation of the Payer's Macaulay Duration in an Interest Rate Swap
#     Based on Bond Math: The Theory Behind the Formulas, Second Edition by
#     Donald J. Smith
#     """
#     # Number of fixed leg payments
#     payment_period = len(swap.fixed_leg.payments)
#
#     # Get coupon frequency
#     coupon_frequency = self.fixed_leg.freq_type.value
#
#     # Get swap rate
#     swap_rate_val = self.swap_rate(value_dt, discount_curve)
#
#     y = swap_rate_val / coupon_frequency
#     N = payment_period
#     c = swap_rate_val / coupon_frequency
#     md1 = (1 + y) / y
#     md2 = 1 + y + (N * (c - y))
#     md3 = c * ((1 + y) ** N - 1) + y
#     mac_duration = 1 - (md1 - md2 / md3)
#     return mac_duration


payment_period = len(swap.fixed_leg.payments) + 1
coupon_frequency = swap.fixed_leg.freq_type.value
swap_rate_val = swap.swap_rate(value_dt, zero_curve)
y = swap_rate_val / coupon_frequency
N = payment_period
c = swap_rate_val / coupon_frequency
md1 = (1 + y) / y
md2 = 1 + y + (N * (c - y))
md3 = c * ((1 + y) ** N - 1) + y
mac_duration = 1 - (md1 - md2 / md3)

delta_mv=-(-mac_duration/4*notional*(-0.004))

In [22]:
swap.value(settle_dt, zero_curve)

-404094.769553849

In [23]:
swap.payer_side_macaulay_duration(value_dt,zero_curve,8)

-6.794198629548404

In [24]:
swap.receiver_side_macaulay_duration(value_dt, zero_curve,8)

6.794198629548404

In [25]:
swap.payer_side_modified_duration(value_dt, zero_curve,8)

-6.743511664816526

In [26]:
swap.receiver_side_modified_duration(value_dt, zero_curve,8)

6.743511664816526

In [27]:
swap.print_payments()

START DATE: 01-APR-2024
MATURITY DATE: 02-JAN-2026
COUPON (%): 3.4000000000000004
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_360_BOND

PAYMENTS SCHEDULE:
+---------+-------------+-------------+-------------+------+----------+------+-----------+
| PAY_NUM |    PAY_dt   |  ACCR_START |   ACCR_END  | DAYS | YEARFRAC | RATE |    PMNT   |
+---------+-------------+-------------+-------------+------+----------+------+-----------+
|    1    | 01-JUL-2024 | 01-APR-2024 | 01-JUL-2024 |  90  |   0.25   | 3.4  |  510000.0 |
|    2    | 01-OCT-2024 | 01-JUL-2024 | 01-OCT-2024 |  90  |   0.25   | 3.4  |  510000.0 |
|    3    | 02-JAN-2025 | 01-OCT-2024 | 02-JAN-2025 |  91  |  0.2528  | 3.4  | 515666.67 |
|    4    | 01-APR-2025 | 02-JAN-2025 | 01-APR-2025 |  89  |  0.2472  | 3.4  | 504333.33 |
|    5    | 01-JUL-2025 | 01-APR-2025 | 01-JUL-2025 |  90  |   0.25   | 3.4  |  510000.0 |
|    6    | 01-OCT-2025 | 01-JUL-2025 | 01-OCT-2025 |  90  |   0.25   | 3.4  |  510000.0 |
| 

In [28]:
swap.print_float_leg_pv()

START DATE: 01-APR-2024
MATURITY DATE: 02-JAN-2026
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_360_BOND

PAYMENTS VALUATION:
+---------+-------------+----------+--------+-----------+--------+-----------+------------+
| PAY_NUM |    PAY_dt   | NOTIONAL |  IBOR  |    PMNT   |   DF   |     PV    |   CUM_PV   |
+---------+-------------+----------+--------+-----------+--------+-----------+------------+
|    1    | 01-JUL-2024 | 60000000 | 0.7529 | 112935.42 | 0.9981 | 112722.26 | 112722.26  |
|    2    | 01-OCT-2024 | 60000000 | 1.6379 | 245686.53 | 0.994  | 244222.77 | 356945.03  |
|    3    | 02-JAN-2025 | 60000000 | 2.7131 | 411480.71 | 0.9873 | 406243.16 | 763188.19  |
|    4    | 01-APR-2025 | 60000000 | 3.5062 | 520087.02 | 0.9788 | 509054.52 | 1272242.71 |
|    5    | 01-JUL-2025 | 60000000 | 3.588  | 538192.75 | 0.9701 | 522093.07 | 1794335.78 |
|    6    | 01-OCT-2025 | 60000000 | 4.293  | 643955.16 | 0.9598 |  618058.3 | 2412394.08 |
|    

In [29]:
swap.print_fixed_leg_pv()

START DATE: 01-APR-2024
MATURITY DATE: 02-JAN-2026
COUPON (%): 3.4000000000000004
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.THIRTY_360_BOND

PAYMENTS VALUATION:
+---------+-------------+----------+------+-----------+--------+-----------+------------+
| PAY_NUM |    PAY_dt   | NOTIONAL | RATE |    PMNT   |   DF   |     PV    |   CUM_PV   |
+---------+-------------+----------+------+-----------+--------+-----------+------------+
|    1    | 01-JUL-2024 | 60000000 | 3.4  |  510000.0 | 0.9981 |  509037.4 |  509037.4  |
|    2    | 01-OCT-2024 | 60000000 | 3.4  |  510000.0 | 0.994  | 506961.51 | 1015998.91 |
|    3    | 02-JAN-2025 | 60000000 | 3.4  | 515666.67 | 0.9873 | 509102.98 | 1525101.88 |
|    4    | 01-APR-2025 | 60000000 | 3.4  | 504333.33 | 0.9788 | 493635.02 | 2018736.9  |
|    5    | 01-JUL-2025 | 60000000 | 3.4  |  510000.0 | 0.9701 | 494743.69 | 2513480.58 |
|    6    | 01-OCT-2025 | 60000000 | 3.4  |  510000.0 | 0.9598 | 489490.19 | 3002970.77 |
|    7    

In [30]:
swap.payer_side_profits(value_dt, zero_curve,8,-0.004)

AttributeError: 'IborSwap' object has no attribute 'payer_side_profits'